In [6]:
import pandas as pd
import os
import json
from dotenv import load_dotenv
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.messages import HumanMessage

# Configuration
load_dotenv()
llm = ChatMistralAI(model="mistral-small-latest", mistral_api_key=os.getenv("MISTRAL_API_KEY"))

def extract_parameters(user_input):
    """Extrait les paramètres structurés depuis l'input utilisateur"""
    
    prompt = f"""
    Extrait les paramètres musicaux de cette demande:
    "{user_input}"
    
    Retourne un objet JSON avec:
    - genre: (string) le genre musical ou null
    - tempo: (number) le tempo en BPM ou null
    - key: (string) la tonalité (ex: "C", "Am", "F#") ou null
    - duration: (number) la durée en minutes ou null
    
    IMPORTANT: Retourne uniquement le JSON valide, rien d'autre.
    Exemple: {{"genre": "rock", "tempo": 120, "key": "Am", "duration": null}}
    """
    
    # Utilisation de l'IA LangChain
    response = llm.invoke([HumanMessage(content=prompt)]).content
    
    try:
        # Nettoyer la réponse
        response_text = response.strip()
        response_text = response_text.replace('```json', '').replace('```', '').strip()
        
        # Extraire le JSON (au cas où il y aurait du texte autour)
        import re
        json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
        if json_match:
            params = json.loads(json_match.group())
        else:
            params = json.loads(response_text)
        
        return params
    except:
        # Fallback simple
        return {"genre": None, "tempo": None, "key": None, "duration": None}

def get_params(genre=None, tempo=None):
    """Trouve les paramètres optimaux"""
    df = pd.read_csv('dataset.csv')
    top_songs = df[df['popularity'] >= df['popularity'].quantile(0.8)]
    
    filtered = top_songs.copy()
    if genre:
        genre_songs = filtered[filtered['track_genre'] == genre]
        if len(genre_songs) > 10:
            filtered = genre_songs
    
    if tempo:
        filtered = filtered[(filtered['tempo'] >= tempo-10) & (filtered['tempo'] <= tempo+10)]
        if len(filtered) < 5:
            filtered = top_songs
    
    params = filtered[['danceability', 'energy', 'key', 'mode', 'acousticness', 
                       'valence', 'tempo', 'duration_ms']].median()
    
    if tempo:
        params['tempo'] = tempo
    
    return params.to_dict(), len(filtered)

def format_params(params):
    """Formatage simple des paramètres"""
    key_names = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    
    return {
        "Tonalité": f"{key_names[int(params['key'])]} {'majeur' if params['mode'] > 0.5 else 'mineur'}",
        "Tempo": f"{params['tempo']:.0f} BPM",
        "Énergie": f"{params['energy']*100:.0f}%",
        "Danse": f"{params['danceability']*100:.0f}%",
        "Humeur": f"{params['valence']*100:.0f}%",
        "Durée": f"{params['duration_ms']/60000:.1f} min"
    }

# Interface principale
print("=== Générateur de Paramètres Musicaux ===")
user_input = input("\nQuelle musique veux-tu créer ? ")

# Extraction structurée par Mistral
params_input = extract_parameters(user_input)
print(f"\nParamètres extraits:")
for k, v in params_input.items():
    if v is not None:
        print(f"  {k}: {v}")

genre = params_input.get('genre')
tempo = params_input.get('tempo')

# Obtenir les paramètres optimaux
params, sample_size = get_params(genre, tempo)
formatted = format_params(params)

# Afficher les résultats
print(f"\nParamètres optimaux ({sample_size} morceaux similaires):")
for k, v in formatted.items():
    print(f"  • {k}: {v}")

# Génération des conseils
prompt = f"""
Donne 3 conseils pour composer:
- Genre: {genre if genre else 'populaire'}
- Tempo: {params['tempo']:.0f} BPM
- Tonalité: {formatted['Tonalité']}
- Humeur: {params['valence']:.2f}/1.0

Conseils concis sur:
1. Rythme à {params['tempo']:.0f} BPM
2. Accords en {formatted['Tonalité']}
3. Ambiance générale
"""

advice = llm.invoke([HumanMessage(content=prompt)]).content
print(f"\nConseils:\n{advice}")

=== Générateur de Paramètres Musicaux ===

Paramètres extraits:
  genre: metal

Paramètres optimaux (546 morceaux similaires):
  • Tonalité: F# majeur
  • Tempo: 125 BPM
  • Énergie: 88%
  • Danse: 48%
  • Humeur: 40%
  • Durée: 4.0 min

Conseils:
1. **Rythme à 125 BPM** :
   - Utilise des **gros temps** (croches ou doubles croches) pour un groove puissant.
   - Ajoute des **syncopes** ou des **triplets** pour dynamiser le rythme.
   - Travaille les **breaks** (ex. : pause rythmique sur un couplet).

2. **Accords en F# majeur** :
   - Base-toi sur **F#m** (mineur) pour une ambiance sombre et mélancolique.
   - Utilise des **accords suspendus** (F#sus4, Bsus2) pour une tension subtile.
   - Expérimente avec des **accords étendus** (F#7, B7) pour un côté plus progressif.

3. **Ambiance générale** :
   - **Dynamique contrastée** : passages calmes (arpegios, voix claire) suivis d’explosions (distorsion, double pédale).
   - **Effets atmosphériques** : reverb, delay et harmonies vocales pou